<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use Spark and Python to Predict Customer Churns</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook contains steps and code to get data, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to the Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache Spark 2.3.

You will use the available data set, **CustomerChurn-WS.csv**, which details anonymous customer and churn information. Use the details of this data set to predict customer churn's risk. 

## Learning goals

You will learn how to:

-  Load a CSV file into an Apache Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache Spark machine learning pipeline.
-  Train and evaluate a model.
-  Store a pipeline and model in Watson Machine Learning (WML) repository.
-  Deploy a model for online scoring using the Watson Machine Learning (WML) API.
-  Score sample scoring data using the WML API.

## Contents

This notebook contains the following parts:

1.	[Set up](#setup)
2.	[Load and explore data](#load)
3.	[Create a spark ml model](#model)
4.	[Store the model](#persistence)
5.	[Predict locally](#prediction)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)
    
**If you are new to data science check out this link:** https://developer.ibm.com/articles/ba-intro-data-science-1/

<a id="setup"></a>
## 1. Set up

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a lite plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)
-  Make sure that you are using a Spark 2.3 kernel
-  Download the **Customer Churn data** (code provided below)

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then use spark *read* method to read data. 

In [ ]:
# Install wget if you don't already have it.
!pip install wget

In [ ]:
import wget

link_to_data = 'https://raw.githubusercontent.com/huisim/DataDSX/master/CustomerChurn-WS.csv'
filename = wget.download(link_to_data)

print(filename)

The csv file **CustomerChurn-WS.csv** is now availble. Load the file to Apache Spark DataFrame using code below.

In [ ]:
#spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache Spark DataFrame methods:
-  print schema
-  remove spaces within column names
-  print top 20 records
-  count all records

In [ ]:
df_data.printSchema()

As you can see, the data contains 9 fields. CHURN field is the one you would like to predict (label).

In [ ]:
# Remove spaces from columns names, it's an example of data preparation that you may have to do before creating a model
df_data = df_data.withColumnRenamed("Est Income", "EstIncome")

In [ ]:
# Print out the top 20 records
df_data.show()

As you can see, the space in the 'Est Income' has been removed. The new column name is 'EstIncome'. 

In [ ]:
# Count all records
print("Number of records: " + str(df_data.count()))

As per expected, the data set contains 2066 records.

The next step provides a quick way for data scientist to review and evaluate the quality of data before modeling. 
Data preparation and data understanding are the most time-consuming tasks in the data mining process.  

In [ ]:
# Statistics summary 
df_data.toPandas().describe()

<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)


**For more information on Apache Spark machine learning check out the following links:**
- https://spark.apache.org/docs/2.3.0/ml-guide.html
- https://spark.apache.org/docs/2.3.0/ml-pipeline.html

### 3.1 Prepare data<a id="prep"></a>

In this subsection you will split your data into: 
- train data set
- test data set
- predict data set

In [ ]:
# Split data into train(70%), test(20%) and predict(10%) datasets
train_data, test_data, predict_data = df_data.randomSplit([0.7,0.2,0.1], seed=6)
train_data.cache()
test_data.cache()
predict_data.cache()

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

As you can see your data has been successfully split into three data sets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In the first step, you need to import the Apache Spark machine learning packages that will be needed in the subsequent steps.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [ ]:
# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
SI1 = StringIndexer(inputCol='Gender', outputCol='GenderEncoded')
SI2 = StringIndexer(inputCol='Status',outputCol='StatusEncoded')
SI3 = StringIndexer(inputCol='LocalBilltype',outputCol='LocalBilltypeEncoded')
SI4 = StringIndexer(inputCol='LongDistanceBilltype',outputCol='LongDistanceBilltypeEncoded')
labelIndexer = StringIndexer(inputCol='CHURN', outputCol='label').fit(df_data)

# Pipelines API requires that input variables are passed in a vector
# In the following step, create a feature vector by combining all features together.
assembler = VectorAssembler(inputCols=["GenderEncoded", "StatusEncoded", "LocalBilltypeEncoded", \
                                       "LongDistanceBilltypeEncoded", "EstIncome", "Age", "Usage"], \
                            outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, convert the indexed labels back to original labels.

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

Now build the pipeline. A pipeline consists of transformers and an estimator.

In [ ]:
pipeline_rf = Pipeline(stages=[SI1,SI2,SI3,SI4,labelIndexer, assembler, rf, labelConverter])

### 3.3 Train the model<a id="train"></a>

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [ ]:
train_data.printSchema()

In [ ]:
model_rf = pipeline_rf.fit(train_data)

In [ ]:
model_rf.transform(test_data)

You can check your **model accuracy** now. Use **test data** to evaluate the model.

In [ ]:
results_rf = model_rf.transform(test_data)
results_rf =results_rf.select(results_rf["ID"],results_rf["CHURN"],results_rf["label"],results_rf["predictedLabel"],results_rf["prediction"],results_rf["probability"])
results_rf.toPandas().head(6)

In [ ]:
# Model Evaluation
'Precision model = {:.2f}.'.format(results_rf.filter(results_rf.label == results_rf.prediction).count() / float(results_rf.count()))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
'Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results_rf))

You can tune your model now to achieve better accuracy. For simplicity, the tuning example is omitted in this example.

<a id="persistence"></a>
## 4. Store the model in the WML repository

In this section you will learn how to use Python client libraries to store your pipeline and model in WML repository.

- [4.1 Import the libraries](#lib)
- [4.2 Save pipeline and model](#save)
- [4.3 Load the model](#load)

### 4.1 Import the libraries<a id="lib"></a>

First, you must install and import the `watson-machine-learning-client` libraries.

**Note**: Python 3.5 and Apache Spark 2.3 is required.

In [ ]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on IBM Cloud. 

If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [ ]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "ad02bdee-1526-4813-9180-f5906e021656",
  "password": "c46fdf43-1ae7-4b44-8483-b3b4702183e0",
  "instance_id": "45c8ab27-1720-46a0-8ec7-75d59af433e4"
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 4.2 Save the pipeline and model<a id="save"></a>

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [ ]:
published_model_details = client.repository.store_model(model=model_rf, meta_props={client.repository.ModelMetaNames.NAME:'Customer Churn Prediction model'}, training_data=train_data, pipeline=pipeline_rf)

In [ ]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

Get saved model metadata from Watson Machine Learning.

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

In [ ]:
client.repository.ModelMetaNames.show()

In [ ]:
# You can get the metadata of stored model 
print(client.repository.get_model_details(model_uid))

### 4.3 Load the model<a id="load"></a>

In this subsection you will learn how to load a saved model from a specified instance of Watson Machine Learning.

In [ ]:
loaded_model = client.repository.load(model_uid)

<a id="prediction"></a>
## 5. Predict locally 

In this section you will learn how to score test data using the loaded model and visualize the prediction results with the plotly package.

- [5.1 Make a local prediction using previously loaded model and test data](#local)

### 5.1 Make a local prediction using previously loaded model and test data<a id="local"></a>

In this subsection you will score the *predict_data* data set.

In [ ]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [ ]:
predictions.show(5)

In [ ]:
# Converting to Pandas data
predictions.toPandas().head(6)

By tabulating a count, you can see which product line is the most popular.

In [ ]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record using the `watson-machine-learning-client`.

**Note:** You can also use the REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

#### Create online deployment for published model.

In [ ]:
deployment_details = client.deployments.create(model_uid, name='Customer Churn Risk Model Deployment')

#### Create an online scoring endpoint. 

In [ ]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

Now, you can send new scoring records (new data) for which you would like to get predictions. To do that, run the following sample code: 

In [ ]:
payload_scoring = {"fields": ["GENDER","STATUS","EstIncome","Age", "LOCALBILLTYPE", "LONGDISTANCEBILLTYPE", "Usage", \
                              "GenderEncoded", "StatusEncoded", "LocalBilltypeEncoded", "LongDistanceBilltypeEncoded"], \
                   "values": \
                             [["M","S", 25000.33, 23,"Budget", "Standard", 100, 1.0, 1.0, 0.0, 0.0]]}

client.deployments.score(scoring_url, payload_scoring)

As you can see we predict that the 23 year old male who is single with estmated income AUD25000.33 who is currently on \'Budget\' local bill type and \'Standard\' long distance bill type with AUD100 usage is about 66\% not likely to churn. 

<a id="summary"></a>
## 7. Summary and next steps     

You successfully completed this notebook! 
 
You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>